# Eksplorasi dan Analisis Data Penawaran Harga Rumah 

# Penjelasan Mengenai Dataset Properti di Bandung

Dataset ini terdiri dari dua versi, yaitu **dataset yang sudah dibersihkan (cleaned dataset)** dan **dataset mentah (raw dataset)**. Kedua dataset ini diperoleh melalui proses web scraping dari situs www.rumah123.com pada bulan Maret 2024 dan fokus pada properti rumah di Bandung, Jawa Barat, Indonesia.

### 1. Dataset yang Sudah Dibersihkan
Dataset yang sudah dibersihkan berisi data yang lebih terstruktur dan telah melalui proses pembersihan untuk menghilangkan kesalahan atau data yang tidak relevan. Atribut yang ada dalam dataset ini meliputi:

- **Nama Rumah**: Nama atau judul dari properti rumah.
- **Lokasi**: Lokasi properti di Bandung.
- **Jumlah Kamar Tidur**: Jumlah kamar tidur dalam rumah.
- **Jumlah Kamar Mandi**: Jumlah kamar mandi yang tersedia.
- **Jumlah Carport**: Jumlah tempat parkir mobil.
- **Harga**: Harga rumah dalam Rupiah (IDR).
- **Luas Tanah**: Luas tanah dalam meter persegi.
- **Luas Bangunan**: Luas bangunan rumah dalam meter persegi.

### 2. Dataset Mentah
Dataset mentah berisi data yang belum diproses dan mungkin mengandung informasi yang tidak konsisten. Selain atribut yang sama seperti pada dataset yang sudah dibersihkan, dataset mentah juga mencakup beberapa atribut tambahan, seperti:

- **Tipe Properti**: Jenis atau kategori properti (misalnya rumah, apartemen).
- **Status**: Status properti (misalnya tersedia, sudah terjual).
- **Cicilan**: Detail mengenai opsi pembayaran cicilan untuk properti.

### 3. Lokasi
- **Kota**: Bandung
- **Provinsi**: Jawa Barat
- **Negara**: Indonesia

### 4. Tujuan Dataset
Dataset ini bertujuan untuk mendukung penelitian dan analisis di bidang real estate, terutama analisis regresi harga rumah di Bandung. Dataset yang sudah dibersihkan memberikan data yang lebih terstruktur dan siap untuk analisis, sedangkan dataset mentah menyediakan data yang lebih lengkap dengan atribut tambahan yang memungkinkan pembersihan dan pemrosesan lebih lanjut.

Dengan kedua versi dataset ini, peneliti, ilmuwan data, dan analis dapat memilih dataset yang paling sesuai dengan kebutuhan mereka, apakah untuk analisis eksploratori, pemodelan prediktif, atau penelitian terkait harga properti. Dataset ini juga mengisi kekosongan dalam ketersediaan data terkait regresi harga rumah di Bandung, Jawa Barat.

### Kesimpulan
Dataset properti rumah di Bandung ini memberikan gambaran yang lebih lengkap mengenai pasar properti di kota tersebut. Dataset yang sudah dibersihkan siap digunakan untuk analisis lebih lanjut, sementara dataset mentah memberikan informasi tambahan yang bisa diproses untuk memperoleh wawasan yang lebih dalam. Dengan adanya dataset ini, diharapkan dapat membantu penelitian terkait pasar properti dan harga rumah di Bandung.

# 1.1 Mempersiapkan Data

Kode berikut menyediakan modul dan opsi yang digunakan dalam notebook ini. File konfigurasi lingkungan analisis khusus (*.yaml*) disertakan dalam folder \Environment yang terdapat dalam repositori ini.

In [19]:
# data wrangling, inspecting and utilities modules
import pandas as pd
import numpy as np
import itertools
import re
import warnings
# statistical analysis modules
from scipy.stats.mstats import mquantiles_cimj, median_cihs
from scipy.stats import bootstrap
# informative chart modules
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.graph_objs import Scattermapbox, Layout, Figure
import plotly.express as px
# data transformation modules
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.base import BaseEstimator, TransformerMixin
# modules automating transformation flow
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
from joblib import dump


In [20]:
# Load the data 
house_df = pd.read_csv(r'/workspaces/Belajar-Machine-Learning-Dasar/Belajar/data/results.csv')

## 1.2. Fitur Dataset 
Bab ini menjelaskan features atau variables yang disediakan dalam dataset.

In [16]:
features = house_df.sample(1).T.rename_axis('features', axis='columns')

features

features,3132
Rumah,Rumah
Premier,NaN
"Rp 2,1 Miliar","Rp 2,7 Miliar"
Cicilan: 7 Jutaan per bulan,Cicilan: 10 Jutaan per bulan
"Rumah 2 Lantai Bagus Sertifikat Hak Milik di Maltra Residence, Bandung",Rumah Bagus Siap Huni Berlokasi di Buah Batu K...
"Andir, Bandung","Buah Batu, Bandung"
3,8
2,6
2.1,1
137 m²,563 m²


### Penjelasan Fitur dalam Dataset Properti di Bandung

Dataset ini berisi sejumlah fitur atau variabel yang menggambarkan properti rumah di Bandung. Setiap fitur mewakili karakteristik tertentu dari properti yang dapat digunakan untuk analisis atau penelitian lebih lanjut. Berikut adalah penjelasan singkat mengenai fitur-fitur yang disediakan dalam dataset:

#### 1. **Nama Rumah**
   - Menyediakan deskripsi singkat atau nama properti, seperti:
     - *"Rumah 2 Lantai Bagus Sertifikat Hak Milik di Maltra Residence, Bandung"*
     - *"Rumah Bagus Siap Huni Berlokasi di Buah Batu, Bandung"*

#### 2. **Lokasi**
   - Menunjukkan lokasi properti di dalam kota Bandung, termasuk area atau kecamatannya. Contoh:
     - *Andir, Bandung*
     - *Buah Batu, Bandung*

#### 3. **Jumlah Kamar Tidur (Bedroom Count)**
   - Menyebutkan jumlah kamar tidur yang tersedia di rumah tersebut.
     - Contoh: 3 kamar tidur, 8 kamar tidur.

#### 4. **Jumlah Kamar Mandi (Bathroom Count)**
   - Menginformasikan jumlah kamar mandi yang tersedia.
     - Contoh: 2 kamar mandi, 6 kamar mandi.

#### 5. **Jumlah Carport (Carport Count)**
   - Menunjukkan jumlah area parkir mobil atau carport yang tersedia.
     - Contoh: 2 carport, 1 carport.

#### 6. **Harga Properti**
   - Memberikan informasi mengenai harga properti dalam Rupiah (IDR).
     - Contoh: Rp 2,1 Miliar, Rp 2,7 Miliar.

#### 7. **Cicilan**
   - Menyediakan informasi tentang estimasi cicilan bulanan jika properti dibeli dengan sistem pembayaran cicilan.
     - Contoh: *"Cicilan: 7 Jutaan per bulan"*, *"Cicilan: 10 Jutaan per bulan"*.

#### 8. **Luas Tanah (Land Area)**
   - Menunjukkan luas tanah dari properti dalam satuan meter persegi (m²).
     - Contoh: 137 m², 563 m².

#### 9. **Luas Bangunan (Building Area)**
   - Menyediakan informasi mengenai luas bangunan properti dalam meter persegi (m²).
     - Contoh: 170 m², 400 m².

---

#### Kesimpulan
Fitur-fitur yang ada dalam dataset ini memberikan gambaran lengkap mengenai properti rumah di Bandung. Mulai dari informasi deskriptif seperti nama rumah dan lokasi, hingga spesifikasi teknis seperti jumlah kamar, harga, luas tanah, dan luas bangunan. Dengan data ini, analisis seperti penentuan harga berdasarkan lokasi, ukuran rumah, atau jumlah kamar dapat dilakukan dengan lebih mudah.

### Tipe Data Setiap Fitur

Setiap variabel dalam dataset sebaiknya direpresentasikan menggunakan tipe data yang sesuai, seperti integer, float, string, atau kategorikal. Berikut adalah pendekatan yang akan digunakan:

- Fitur-fitur yang merepresentasikan jumlah atau kuantitas tertentu (seperti jumlah carport yang tersedia) lebih cocok direpresentasikan sebagai bilangan bulat. Untuk kemudahan analisis, tipe data dari masing-masing fitur akan ditentukan dengan jelas.  
- Beberapa fitur memiliki tipe data kategorikal, seperti *certificate* (sertifikat), sementara beberapa kategori lain memiliki hierarki atau urutan alami, seperti *electricity* (kelistrikan).

In [22]:
house_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7610 entries, 0 to 7609
Data columns (total 11 columns):
 #   Column                                                                  Non-Null Count  Dtype 
---  ------                                                                  --------------  ----- 
 0   Rumah                                                                   7610 non-null   object
 1   Premier                                                                 144 non-null    object
 2   Rp 2,1 Miliar                                                           7610 non-null   object
 3   Cicilan: 7 Jutaan per bulan                                             7610 non-null   object
 4   Rumah 2 Lantai Bagus Sertifikat Hak Milik di Maltra Residence, Bandung  7610 non-null   object
 5   Andir, Bandung                                                          7610 non-null   object
 6   3                                                                       7610 non-null   

In [ ]:
# View data types
house_df.dtypes

In [ ]:
# Split data into numerical and categorical types
numerical_cols = house_df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = house_df.select_dtypes(include=['object', 'category']).columns

print('Numerical columns:', numerical_cols)
print('Categorical columns:', categorical_cols)